## Part-4.5：Chain -> TransformChain：实现快捷处理超长文本


In [ ]:
import os
import openai
import configparser
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import DatetimeOutputParser
from langchain import LLMChain
from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain

conf = configparser.ConfigParser()
current_directory = os.path.dirname(os.path.realpath('__file__'))
config_file_path = os.path.join(current_directory, '..', '..', '..', 'config.ini')
conf.read(config_file_path)
api_key = conf.get("Openai", "api_key")  # 在config.ini中配置自己的APIkey
os.environ["HTTP_PROXY"] = conf.get("Proxy", "HTTP_PROXY")  # 配置自己的代理
os.environ["HTTPS_PROXY"] = conf.get("Proxy", "HTTPS_PROXY")
chat_model = "gpt-3.5-turbo"
text_model = "text-davinci-003"

In [ ]:
llm = OpenAI(openai_api_key=api_key)

In [ ]:
from langchain.chains import TransformChain

with open("../datas/the_old_man_and_the_sea.txt") as f:
    novel_text = f.read()


def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text": shortened_text}


transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)
transformed_novel = transform_chain(novel_text)

In [ ]:
transformed_novel.keys()

In [ ]:
template = """总结下面文本:

{output_text}

总结:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

simple_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

simple_chain.run(novel_text[:100])